## Convert and combine data

In [26]:
import pandas as pd
df = pd.read_csv("data.csv")
df = df.drop(index=0)

In [27]:
import glob
import os
import shutil
inks1 = glob.glob("datawrite/*.ink")
inks1Dataframe = pd.DataFrame(inks1, columns=["path"])
inks1Dataframe["name"] = inks1Dataframe["path"].apply(lambda x: os.path.basename(x).replace(".ink", ""))
inks1Dataframe.head()

,path,name
0,datawrite\00000704.ink,00000704
1,datawrite\00005443.ink,00005443
2,datawrite\00143892.ink,00143892
3,datawrite\00291041.ink,00291041
4,datawrite\00318458.ink,00318458


In [28]:
df["name"] = df["file_name"].apply(lambda x: os.path.basename(x).replace(".png", ""))
df.head()

,Unnamed: 0,id,file_name,Latex,name
1,1,0,Data\GenData\45348833.png,\sum_{\alpha}^{\sin{b}}{\tan{+{\sin{\alpha}}}},45348833
2,2,1,Data\GenData\10474313.png,\sum_{\tan{-{+{\pi}}}}^{\prod_{x}^{\beta}{\pro...,10474313
3,3,2,Data\GenData\77236513.png,{\ln{{{2}\times{z}}-{\exp{\alpha}}}}_{\prod_{\...,77236513
4,4,3,Data\GenData\45582352.png,{\pi}\times{y},45582352
5,5,4,Data\GenData\58611711.png,\prod_{e}^{\pi}{{{y}\cdot{x}}\cdot{\alpha}},58611711


In [29]:
dfCombined = pd.merge(df, inks1Dataframe, on="name", how="left")
dfCombined

,Unnamed: 0,id,file_name,Latex,name,path
0,1,0,Data\GenData\45348833.png,\sum_{\alpha}^{\sin{b}}{\tan{+{\sin{\alpha}}}},45348833,datawrite\45348833.ink
1,2,1,Data\GenData\10474313.png,\sum_{\tan{-{+{\pi}}}}^{\prod_{x}^{\beta}{\pro...,10474313,datawrite\10474313.ink
2,3,2,Data\GenData\77236513.png,{\ln{{{2}\times{z}}-{\exp{\alpha}}}}_{\prod_{\...,77236513,datawrite\77236513.ink
3,4,3,Data\GenData\45582352.png,{\pi}\times{y},45582352,datawrite\45582352.ink
4,5,4,Data\GenData\58611711.png,\prod_{e}^{\pi}{{{y}\cdot{x}}\cdot{\alpha}},58611711,datawrite\58611711.ink
...,...,...,...,...,...,...
1995,1996,1995,Data\GenData\60960126.png,{x}\cdot{+{\sin{{b}_{1}}}},60960126,NaN
1996,1997,1996,Data\GenData\07532053.png,\binom{\pi}{\binom{x}{\log{{z}\cdot{\beta}}}},07532053,datawrite\07532053.ink
1997,1998,1997,Data\GenData\22405761.png,\int_{1}^{\tan{2}}{\prod_{\beta}^{a}{{\sin{\pi...,22405761,NaN
1998,1999,1998,Data\GenData\09095631.png,{x}-{x},09095631,datawrite\09095631.ink


In [30]:
dfCombined["data_source"] = "Datagen"
dfCombined["tags"] = "train"

In [31]:
dfCombined.to_csv("dataCombined.csv", index=False)

In [32]:
dfCombined[dfCombined["path"].isnull()].count()

Unnamed: 0     387
id             387
file_name      387
Latex          387
name           387
path             0
data_source    387
tags           387
dtype: int64

In [32]:
with open("CROHME/crohme2019_train.txt", "r") as f:
    for line in f:
        line = line.strip()
        line = line.split("\t")
        print(line)

['crohme2019/train/MfrDB2372.inkml', 'f Sup \\prime NoRel ( Right x Right ) Right = NoRel 1 NoRel - Below 2 Right \\sqrt Inside x']
['crohme2019/train/MfrDB3179.inkml', 'h Right ( Right x Right COMMA Right y Right ) Right = Right x Sup y Right + NoRel 1 NoRel - Below y']
['crohme2019/train/200922-949-204.inkml', 'F Right f Right N']
['crohme2019/train/MfrDB1321.inkml', 'x Sup 2 NoRel - Right x Right - Right 6 Right \\lt Right 0']
['crohme2019/train/MfrDB3271.inkml', '3 Right a Sup 2 NoRel b Sup 3 NoRel + Right 5 Right a Sup 3 NoRel b Sup 2 NoRel - NoRel a Sup 5 NoRel b Sup 8 NoRel - Below 2']
['crohme2019/train/KME2G3_26_sub_22.inkml', 'x Sub x NoRel x NoRel + Right y Sub y NoRel y NoRel + Right z Sub z NoRel z NoRel - Right x Right - Right y Right - Right z']
['crohme2019/train/TrainData2_7_sub_63.inkml', 'a NoRel - Above 1 NoRel F Right ( Right a Right x Right + Right b Right ) Right + Right C']
['crohme2019/train/200923-1556-13.inkml', '\\sqrt Inside 1 Right + Right \\sqrt Inside 1 

In [34]:
# Combine CROHME data into dfCombined (with correct tags)
import pandas as pd
import os

def read_crohme_txt(txt_path, tag):
    data = []
    with open(txt_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split("\t")
            if len(parts) == 2:
                data.append({"name": parts[0], "Latex": parts[1], "tags": tag})
    return pd.DataFrame(data)

crohme_files = [
    ("CROHME/crohme2019_train.txt", "train"),
    ("CROHME/crohme2019_valid.txt", "valid"),
    ("CROHME/crohme2019_test.txt", "test")
]
df_crohme = pd.concat([read_crohme_txt(f, tag) for f, tag in crohme_files], ignore_index=True)
df_crohme["path"] = df_crohme["name"]  # path = file name
# Add missing columns to match dfCombined
for col in ["file_name", "data_source"]:
    if col not in df_crohme:
        df_crohme[col] = ""
df_crohme["data_source"] = "CROHME"
# Reorder columns to match dfCombined
cols = list(dfCombined.columns)
df_crohme = df_crohme.reindex(columns=cols)
# Combine
new_dfCombined = pd.concat([dfCombined, df_crohme], ignore_index=True)
new_dfCombined.to_csv("dataCombined_with_crohme.csv", index=False)
new_dfCombined.head()

,Unnamed: 0,id,file_name,Latex,name,path,data_source,tags
0,1.0,0,Data\GenData\45348833.png,\sum_{\alpha}^{\sin{b}}{\tan{+{\sin{\alpha}}}},45348833,NaN,Datagen,train
1,2.0,1,Data\GenData\10474313.png,\sum_{\tan{-{+{\pi}}}}^{\prod_{x}^{\beta}{\pro...,10474313,NaN,Datagen,train
2,3.0,2,Data\GenData\77236513.png,{\ln{{{2}\times{z}}-{\exp{\alpha}}}}_{\prod_{\...,77236513,NaN,Datagen,train
3,4.0,3,Data\GenData\45582352.png,{\pi}\times{y},45582352,NaN,Datagen,train
4,5.0,4,Data\GenData\58611711.png,\prod_{e}^{\pi}{{{y}\cdot{x}}\cdot{\alpha}},58611711,NaN,Datagen,train


In [ ]:
# Function to convert .inkml file to .png
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import os

def inkml_to_png(inkml_path, out_path=None, color='black', width=2, figsize=(4,4)):
    """
    Convert .inkml (XML) file to PNG image.
    - inkml_path: path to .inkml file
    - out_path: path to .png file (if None, will use same name as .inkml)
    - color: line color
    - width: line width
    - figsize: matplotlib figure size
    """
    tree = ET.parse(inkml_path)
    root = tree.getroot()
    traces = root.findall('.//trace')
    plt.figure(figsize=figsize)
    for trace in traces:
        coords = trace.text.strip().split()
        points = [tuple(map(float, pt.split(','))) for pt in coords if ',' in pt]
        if len(points) >= 2:
            xs, ys = zip(*points)
            plt.plot(xs, ys, color=color, linewidth=width)
    plt.axis('equal')
    plt.axis('off')
    if out_path is None:
        out_path = os.path.splitext(inkml_path)[0] + '.png'
    plt.savefig(out_path, bbox_inches='tight', pad_inches=0, dpi=300)
    plt.close()
    return out_path

# Example usage:
# inkml_to_png('path/to/file.inkml')

In [39]:
# Convert all .inkml files in CROHME folders to .png and update new_dfCombined
import glob
import os

def batch_inkml_to_png(inkml_dir, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    inkml_files = glob.glob(os.path.join(inkml_dir, '*.inkml'))
    name2png = {}
    for inkml_path in inkml_files:
        name = os.path.splitext(os.path.basename(inkml_path))[0]
        out_path = os.path.join(out_dir, name + '.png')
        ink_to_png(inkml_path, out_path)  # reuse the same function, assuming .inkml is similar XML
        name2png[name] = out_path
    return name2png

crohme_sets = [
    (r'CROHME/crohme2019/crohme2019/train', r'CROHME/train'),
    (r'CROHME/crohme2019/crohme2019/valid', r'CROHME/valid'),
    (r'CROHME/crohme2019/crohme2019/test',  r'CROHME/test'),
]
name2png_all = {}
for inkml_dir, out_dir in crohme_sets:
    name2png = batch_inkml_to_png(inkml_dir, out_dir)
    name2png_all.update(name2png)

# Cập nhật trường file_name trong new_dfCombined cho các mẫu CROHME
mask_crohme = new_dfCombined['data_source'] == 'CROHME'
def get_png_path(row):
    if row['data_source'] == 'CROHME' and row['name'] in name2png_all:
        return name2png_all[row['name']]
    return row['file_name']
new_dfCombined['file_name'] = new_dfCombined.apply(get_png_path, axis=1)
# Lưu lại file csv mới
new_dfCombined.to_csv('dataCombined_with_crohme_png.csv', index=False)
new_dfCombined.head()

NameError: name 'ink_to_png' is not defined

## Convert image

In [1]:
# Helper functions for loading and processing CROHME dataset
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw
import os
import re

def parse_inkml(file_path):
    """
    Parse an InkML file to extract strokes and LaTeX ground truth.
    
    Args:
        file_path: Path to the InkML file
    
    Returns:
        tuple: (strokes, latex) where strokes is a list of stroke coordinates
               and latex is the ground truth LaTeX string
    """
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        
        # Define namespace
        ns = {'ink': 'http://www.w3.org/2003/InkML'}
        
        # Extract LaTeX ground truth
        latex = None
        for annotation in root.findall('.//ink:annotation[@type="truth"]', ns):
            latex = annotation.text
            if latex:
                # Remove $ signs if present
                latex = latex.strip()
                if latex.startswith('$') and latex.endswith('$'):
                    latex = latex[1:-1]
                break
        
        # Extract strokes
        strokes = []
        for trace in root.findall('.//ink:trace', ns):
            if trace.text:
                points = []
                coords = trace.text.strip().split(',')
                for coord in coords:
                    parts = coord.strip().split()
                    if len(parts) >= 2:
                        try:
                            x, y = float(parts[0]), float(parts[1])
                            points.append((x, y))
                        except ValueError:
                            continue
                if points:
                    strokes.append(points)
        
        return strokes, latex
    except Exception as e:
        print(f"Error parsing {file_path}: {e}")
        return None, None

def render_inkml_to_image(strokes, img_width=800, img_height=200, padding=10):
    """
    Render InkML strokes as a PIL Image.
    
    Args:
        strokes: List of strokes, where each stroke is a list of (x, y) coordinates
        img_width: Target image width
        img_height: Target image height
        padding: Padding around the strokes
    
    Returns:
        PIL.Image: Rendered image with white background and black strokes
    """
    if not strokes:
        return Image.new('RGB', (img_width, img_height), 'white')
    
    # Find bounding box
    all_x = [x for stroke in strokes for x, y in stroke]
    all_y = [y for stroke in strokes for x, y in stroke]
    
    if not all_x or not all_y:
        return Image.new('RGB', (img_width, img_height), 'white')
    
    min_x, max_x = min(all_x), max(all_x)
    min_y, max_y = min(all_y), max(all_y)
    
    # Calculate scaling to fit in target size with padding
    width = max_x - min_x
    height = max_y - min_y
    
    if width == 0 or height == 0:
        return Image.new('RGB', (img_width, img_height), 'white')
    
    scale_x = (img_width - 2 * padding) / width
    scale_y = (img_height - 2 * padding) / height
    scale = min(scale_x, scale_y)
    
    # Create image
    img = Image.new('RGB', (img_width, img_height), 'white')
    draw = ImageDraw.Draw(img)
    
    # Draw strokes
    for stroke in strokes:
        if len(stroke) < 2:
            continue
        
        # Normalize and scale coordinates
        normalized_stroke = [
            (
                (x - min_x) * scale + padding,
                (y - min_y) * scale + padding
            )
            for x, y in stroke
        ]
        
        # Draw lines between consecutive points
        for i in range(len(normalized_stroke) - 1):
            draw.line(
                [normalized_stroke[i], normalized_stroke[i + 1]],
                fill='black',
                width=2
            )
    
    return img

def load_crohme_dataset(txt_file, base_path="CROHME"):
    """
    Load CROHME dataset from a text file with label graphs.
    
    Args:
        txt_file: Path to the text file (e.g., 'CROHME/crohme2019_train.txt')
                  Format: path/to/file.inkml<TAB>label_graph
        base_path: Base directory containing the CROHME data
    
    Returns:
        list: List of dictionaries with 'image' and 'text' keys
              where 'text' contains the label graph representation
    """
    dataset = []
    
    with open(txt_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            
            # Parse line: "path/to/file.inkml\tlabel_graph"
            parts = line.split('\t')
            if len(parts) != 2:
                # Skip lines without label graph
                continue
            
            inkml_path, label_graph = parts
            full_path = os.path.join(base_path, inkml_path)
            
            # Check if file exists
            if not os.path.exists(full_path):
                continue
            
            # Parse InkML file to get strokes (we don't need LaTeX anymore)
            strokes, _ = parse_inkml(full_path)
            
            if strokes and label_graph:
                # Render strokes to image
                image = render_inkml_to_image(strokes)
                
                dataset.append({
                    'image': image,
                    'text': label_graph  # Use label graph instead of LaTeX
                })
    
    print(f"Loaded {len(dataset)} samples from {txt_file}")
    return dataset

In [2]:
import os
import pandas as pd
from inkml2img import inkml2img

# Đọc dữ liệu
df = pd.read_csv("dataCombined_with_crohme.csv")
crohme_df = df[df["data_source"] == "CROHME"]

# Tạo thư mục lưu ảnh
out_root = "crohme2019"
for tag in ["train", "valid", "test"]:
    os.makedirs(os.path.join(out_root, tag), exist_ok=True)

for idx, row in crohme_df.iterrows():
    # Trường name đã là đường dẫn kiểu crohme2019/train/MfrDB2372.inkml
    inkml_path = "CROHME//crohme2019/"+row["name"]
    tag = row["tags"]
    # Lấy tên file không đuôi
    base = os.path.splitext(os.path.basename(inkml_path))[0]
    out_path = os.path.join(out_root, tag, base + ".png")
    if os.path.exists(inkml_path):
        if os.path.exists(out_path):
            print(f"Already exists: {out_path}")
            continue
        try:
            strokes, _ = parse_inkml(inkml_path)
            if strokes:
                # Render strokes to image
                image = render_inkml_to_image(strokes)
                image.save(out_path)
                print(f"Converted {inkml_path} to {out_path}")
        except Exception as e:
            print(f"Error converting {inkml_path}: {e}")
        
    else:
        print(f"Not found: {inkml_path}")

Converted CROHME//crohme2019/crohme2019/train/MfrDB2372.inkml to crohme2019\train\MfrDB2372.png
Converted CROHME//crohme2019/crohme2019/train/MfrDB3179.inkml to crohme2019\train\MfrDB3179.png
Already exists: crohme2019\train\200922-949-204.png
Converted CROHME//crohme2019/crohme2019/train/MfrDB1321.inkml to crohme2019\train\MfrDB1321.png
Converted CROHME//crohme2019/crohme2019/train/MfrDB3271.inkml to crohme2019\train\MfrDB3271.png
Already exists: crohme2019\train\KME2G3_26_sub_22.png
Already exists: crohme2019\train\TrainData2_7_sub_63.png
Already exists: crohme2019\train\200923-1556-13.png
Already exists: crohme2019\train\2009212-1031-3.png
Already exists: crohme2019\train\formulaire029-equation040.png
Already exists: crohme2019\train\formulaire024-equation066.png
Converted CROHME//crohme2019/crohme2019/train/MfrDB3384.inkml to crohme2019\train\MfrDB3384.png
Already exists: crohme2019\train\formulaire027-equation055.png
Already exists: crohme2019\train\formulaire020-equation062.png
A

## Create vocab file


In [6]:
import pandas as pd
# tokenizer = PreTrainedTokenizerFast(tokenizer_file="D:/projectDAT/image-computer/new_process/Data/tokenizer.json", unk_token="[UNK]", pad_token="[PAD]", cls_token="[CLS]", sep_token="[SEP]", mask_token="[MASK]")
df = pd.read_csv("D:/projectDAT/image-computer/new_process/Data/dataCombined_with_crohme.csv")
df = df[df["data_source"] == "CROHME"].reset_index()
df.head()

,index,Unnamed: 0,id,file_name,Latex,name,path,data_source,tags
0,2000,NaN,NaN,NaN,f Sup \prime NoRel ( Right x Right ) Right = N...,crohme2019/train/MfrDB2372.inkml,crohme2019/train/MfrDB2372.inkml,CROHME,train
1,2001,NaN,NaN,NaN,h Right ( Right x Right COMMA Right y Right ) ...,crohme2019/train/MfrDB3179.inkml,crohme2019/train/MfrDB3179.inkml,CROHME,train
2,2002,NaN,NaN,NaN,F Right f Right N,crohme2019/train/200922-949-204.inkml,crohme2019/train/200922-949-204.inkml,CROHME,train
3,2003,NaN,NaN,NaN,x Sup 2 NoRel - Right x Right - Right 6 Right ...,crohme2019/train/MfrDB1321.inkml,crohme2019/train/MfrDB1321.inkml,CROHME,train
4,2004,NaN,NaN,NaN,3 Right a Sup 2 NoRel b Sup 3 NoRel + Right 5 ...,crohme2019/train/MfrDB3271.inkml,crohme2019/train/MfrDB3271.inkml,CROHME,train


In [2]:
df["token"][1]

'NoRel'

In [2]:
with open("vocab.txt", "w", encoding="utf-8") as f:
    f.write("<PAD>\n<SOS>\n<EOS>\n<UNK>\n")
    for token in df["token"]:
        f.write(f"{token}\n")

In [9]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.normalizers import Sequence, NFD, Lowercase, StripAccents
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing

# 1️⃣ Tạo model WordPiece từ vocab.txt
tokenizer = Tokenizer(WordPiece(vocab="vocab.txt", unk_token="[UNK]"))

# 2️⃣ Thiết lập normalizer (chuẩn hóa)
tokenizer.normalizer = Sequence([NFD(), Lowercase(), StripAccents()])

# 3️⃣ Thiết lập pre-tokenizer (tách từ theo khoảng trắng)
tokenizer.pre_tokenizer = Whitespace()

# 4️⃣ Thiết lập post-processor (thêm token đặc biệt)
tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B [SEP]",
    special_tokens=[
        ("[CLS]", 2),
        ("[SEP]", 3),
    ],
)

# 5️⃣ Lưu ra file tokenizer.json
tokenizer.save("tokenizer.json")
print("✅ Saved tokenizer.json successfully!")


✅ Saved tokenizer.json successfully!


C:\Users\Admin\AppData\Local\Temp\ipykernel_24252\2918212317.py:8: DeprecationWarning: Deprecated in 0.9.0: WordPiece.__init__ will not create from files anymore, try `WordPiece.from_file` instead
  tokenizer = Tokenizer(WordPiece(vocab="vocab.txt", unk_token="[UNK]"))


In [7]:
import numpy as np
A = np.array([[1, 2, 3], [4, 5, 6]])
A[:, 1] = [10, 20]                        

In [1]:
import imagesize
import glob
import os
train = glob.glob("crohme2019/train/*.png")
sizes = [imagesize.get(f) for f in train]
heights = [s[1] for s in sizes]
widths = [s[0] for s in sizes]
max(heights)
max(widths)

515

In [2]:
print(max(heights), max(widths))  # 64, 256

389 515


In [3]:
import imagesize
import glob
import os
train = glob.glob("crohme2019/test/*.png")
sizes = [imagesize.get(f) for f in train]
heights = [s[1] for s in sizes]
widths = [s[0] for s in sizes]

In [5]:
print(max(heights), max(widths))  # 64, 256

389 515


## Convert label graph to latex

In [1]:
import json
with open("tokenizer.json", "r") as f:
    vocab = json.load(f)
    vocabs = vocab["model"]["vocab"]
vocabs = list(vocabs.keys())
relationship_vocabs = ["Right","NoRel","Sup","Sub","Below","Inside","Above","COMMA"]
function_vocabs = ["\\sqrt","\\sin","\\sum","\\int","\\cos","\\log","\\lim"]
special_vocabs = ["<PAD>","<SOS>","<EOS>","<UNK>"]


In [ ]:
class Stack:
  def __init__(self):
    self.stack = []

  def push(self, element):
    self.stack.append(element)

  def pop(self):
    if self.isEmpty():
      return "Stack is empty"
    return self.stack.pop()

  def peek(self):
    if self.isEmpty():
      return "Stack is empty"
    return self.stack[-1]

  def isEmpty(self):
    return len(self.stack) == 0

  def size(self):
    return len(self.stack)

def label_to_latex(label_graph:str)->str:
    labels = label_graph.split()
    latex = ""
    stack = Stack()
    for i, label in enumerate(labels):
        if label == "Right":
            latex += " "
        elif label == "Sub":
            if i < len(labels)-2 and labels[i+2] in relationship_vocabs:
                latex += "_"
            else:
              latex += "_{"
              stack.push("Sub")
        elif label == "Sup":
            if i < len(labels)-2 and labels[i+2] in relationship_vocabs:
                latex += "^"
            else:
              latex += "^{"
              stack.push("Sup")
        elif label == "COMMA":
            latex += ","
        elif label == "Above":
            if labels[i-1] == "-":
              latex = latex[:-1]  # remove last space
              latex += "\\frac{"
              stack.push("\\frac")
        elif label == "Inside":
            latex += "{"
            stack.push("\\sqrt")
        elif label == "Below":
           if labels[i-1] == "\\sum":
              latex += "_{"
              stack.push("Sum")
        elif label == "NoRel":
            if stack.peek() == "\\frac":
               latex += "}{"
               stack.pop()
               stack.push("NoRel")
            elif stack.peek() == "Sum":
                latex += "}"
                stack.pop()
            elif not stack.isEmpty():
                latex += "}"
                stack.pop()
            elif  i<len(labels)-2 and labels[i+1] == "-" and labels[i+2] == "Below":
               last_latex = latex[-1]  # remove last space
               latex = latex[:-1]  # remove last space
               latex += "\\frac{" + last_latex + "}{"
               labels[i+1] = ""  # skip the "-"
               stack.push("Below")
            else:
                latex += " "
        elif label not in relationship_vocabs and label not in function_vocabs:
            latex += label
        elif label == "\\sum":
            latex += "\\sum"
        elif label == "\\int":
            latex += "\\int"
        elif label == "\\sqrt":
            latex += "\\sqrt"
    print(stack.stack)
    while not stack.isEmpty():
      latex += "}"
      stack.pop()

    return latex
label_to_latex("\sum Below a NoRel p Sub a NoRel = Right \sum Right X Sub a NoRel = Right 0 NoRel a Right \sqrt Inside 1 Right 2 Right + 1 Right 5 ")  # "a_{b} + b^{c} d - \frac{c}{d}"

['\\sqrt']


'\\sum_{a}p_a = \\sum X_a = 0 a \\sqrt{1 2 +1 5}'